# Discussion 1: Pandas Review

This section of Discussion 1 is meant to review [Pandas](https://pandas.pydata.org/docs/), one of the most popular Python libraries for data-wrangling. It's a crucial tool in any machine learning researcher or engineer's repertoire and you will continue to use it heavily throughout the semester.

Let's start by loading the necessary packages for today's exercise, which will look at movie data from IMDb. We use the `read_csv` function to load data from the internet, but you can also use this function to load a file from your local storage.

*NOTE: The output of the cells may not be correct if run out of order, even if your code is correct. If in doubt, you can run all cells at once, which should take no more than a few seconds.*


In [34]:
import os, random, numpy as np
SEED = 189

os.environ["PYTHONHASHSEED"] = str(SEED)

random.seed(SEED)
np.random.seed(SEED)

In [35]:
import pandas as pd
import plotly.express as px

# Load the title_basics dataset from IMDb
title_basics  = pd.read_csv(
    "https://datasets.imdbws.com/title.basics.tsv.gz",
    sep="\t", compression="gzip", na_values="\\N", nrows=50000
)
# Load the title_ratings dataset from IMDb
title_ratings = pd.read_csv(
    "https://datasets.imdbws.com/title.ratings.tsv.gz",
    sep="\t", compression="gzip", na_values="\\N", nrows=50000
)

# Sort both dataframes by 'tconst' and reset the index
title_basics  = title_basics.sort_values("tconst", kind="mergesort").reset_index(drop=True)
title_ratings = title_ratings.sort_values("tconst", kind="mergesort").reset_index(drop=True)

## Part 1: Exploration and Data Cleaning

Let's start by inspecting the `title_basics` `DataFrame`.

### Q 1.1

How many columns are in the `title_basics` `DataFrame`?

What is the data type of the startYear column? Does this make sense?


In [36]:

# YOUR CODE HERE
title_basics_numcols=title_basics.shape[1]
print(title_basics_numcols)

startYear_dtype=title_basics['startYear'].dtype
print(startYear_dtype)

9
float64


### Q1.2

What is the value in 101st row of the `primaryTitle` column of the `title_basics` `DataFrame`? *HINT: Recall that* `DataFrame` *uses 0-indexing*


In [37]:

# YOUR CODE HERE
val = title_basics.loc[100, 'primaryTitle']
print(val)


The Beach at Villiers in a Gale


### Q1.3

Display the first 3 rows and the last 6 rows of the `title_basics` `DataFrame` as a single `DataFrame`.


In [38]:

# YOUR CODE HERE
print(title_basics.head(3))

print(title_basics.tail(6))


      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short            Poor Pierrot          Pauvre Pierrot   

   isAdult  startYear  endYear  runtimeMinutes                    genres  
0        0     1894.0      NaN             1.0         Documentary,Short  
1        0     1892.0      NaN             5.0           Animation,Short  
2        0     1892.0      NaN             5.0  Animation,Comedy,Romance  
          tconst titleType        primaryTitle       originalTitle  isAdult  \
49994  tt0050922     movie        The Sad Sack        The Sad Sack        0   
49995  tt0050923     movie     Saddle the Wind     Saddle the Wind        0   
49996  tt0050924     movie  Saeta del ruiseñor  Saeta del ruiseñor        0   
49997  tt0050925     movie        Said effendi        Said effendi        0   
49998  t

### Q1.4

How many unique `titleTypes` are there in the `title_basics` `DataFrame`? Which is the most common?


In [39]:
# YOUR CODE HERE
print(title_basics['titleType'].unique().size)

print(title_basics['titleType'].value_counts().idxmax())


7
movie


Now let's practice some common `DataFrame` modifications.

### Q1.5

Remove the `originalTitle` and `endYear` columns from the `title_basics` `DataFrame`. Make sure that the columns are permanently removed from the `title_basics` `DataFrame`.


In [40]:

# YOUR CODE HERE

title_basics=title_basics.drop(columns=['originalTitle','endYear'])
title_basics.head()

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,0,1894.0,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,0,1892.0,5.0,"Animation,Short"
2,tt0000003,short,Poor Pierrot,0,1892.0,5.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,0,1892.0,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,0,1893.0,1.0,Short


### Q1.6

Rename `primaryTitle` to `title` and `startYear` to `year` in the `title_basics` `DataFrame`. Make sure that the changes are reflected permanently in the `title_basics` `DataFrame`.



In [41]:
# YOUR CODE HERE
title_basics = title_basics.rename(columns={'primaryTitle': 'title', 'startYear': 'year'})

title_basics.head()

,tconst,titleType,title,isAdult,year,runtimeMinutes,genres
0,tt0000001,short,Carmencita,0,1894.0,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,0,1892.0,5.0,"Animation,Short"
2,tt0000003,short,Poor Pierrot,0,1892.0,5.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,0,1892.0,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,0,1893.0,1.0,Short


### Q1.7

A crucial step in most data processing pipelines for machine learning is dealing with missing or corrupted data. Often, these missing values are represented as a `NaN` (not a number).

Sometimes in the context of machine learning we'd want to estimate a value for a missing feature rather than remove that sample point entirely. Can you think of some simple ways in which we could perform that estimation?

[YOUR ANSWER HERE] maybe replace them with the average value of the column

### Q1.8

Remove all rows from the `title_basics` `DataFrame` where `runtimeMinutes` or `year` is `NaN`.

In [42]:
initial_length = title_basics.shape[0]

# YOUR CODE HERE

title_basics = title_basics.dropna(subset=['runtimeMinutes', 'year'])

final_length = title_basics.shape[0]

print(f"{initial_length - final_length} rows removed from dataframe")


9246 rows removed from dataframe


### Q1.9

Change the data type of the `year` column in the `title_basics` `DataFrame` to something that makes more sense. Then confirm that the change is permanently applied.

In [43]:

# YOUR CODE HERE

title_basics['year']=title_basics['year'].astype(int)


Let's practice some more basic filtering and sorting now.

### Q1.10

Extract the feature films (`titleType == "movie"`) released in 1954 from the `title_basics` `DataFrame` (save this as a new `DataFrame`, `feature_films_1954`).

In [44]:
# YOUR CODE HERE

feature_films_1954=title_basics[(title_basics['titleType']=='movie') & (title_basics['year']==1954)]
feature_films_1954.head()

,tconst,titleType,title,isAdult,year,runtimeMinutes,genres
35844,tt0036493,movie,Mystery of the Black Jungle,0,1954,80.0,"Action,Adventure,Mystery"
36914,tt0037585,movie,Knights of the Queen,0,1954,79.0,Adventure
37343,tt0038020,movie,Relato policíaco,0,1954,75.0,Crime
37410,tt0038089,movie,Siluri umani,0,1954,87.0,"Drama,War"
37558,tt0038240,movie,Das Licht der Liebe,0,1954,95.0,Drama


### Q1.11

Among the feature films from 1954, which film has the longest runtime? Return its `title` and `runtimeMinutes` as a `DataFrame` extracted from the `feature_films_1954` `DataFrame`.


In [45]:
# YOUR CODE HERE

feature_films_1954[feature_films_1954['runtimeMinutes']==feature_films_1954['runtimeMinutes'].max()][['title','runtimeMinutes']]

,title,runtimeMinutes
46203,Gunfighters of the Northwest,315.0


## Part 2: Complex Modifications, Aggregations, Merging, and Plotting

Let's first modify the `title_basics` `DataFrame` so that we have one genre in for each row by duplicating rows that have multiple genres. [`df.explode`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html) will be helpful for this transformation.

In [46]:
# Split the genres string into a list of genres
title_basics['genres'] = title_basics['genres'].str.split(',')
# Explode the list of genres into separate rows
title_basics = title_basics.explode('genres')

title_basics.head()

,tconst,titleType,title,isAdult,year,runtimeMinutes,genres
0,tt0000001,short,Carmencita,0,1894,1.0,Documentary
0,tt0000001,short,Carmencita,0,1894,1.0,Short
1,tt0000002,short,Le clown et ses chiens,0,1892,5.0,Animation
1,tt0000002,short,Le clown et ses chiens,0,1892,5.0,Short
2,tt0000003,short,Poor Pierrot,0,1892,5.0,Animation


### Q2.1

For each `genre` in the `title_basics` `DataFrame`, compute the mean runtime of feature films released since 1960.
Show the five longest-mean genres.

In [47]:
# YOUR CODE HERE

title_basics.groupby('genres')['runtimeMinutes'].mean().sort_values(ascending=False).head(5)


genres
Biography    92.994361
History      88.450255
Action       88.001427
Adventure    84.643666
Thriller     82.176090
Name: runtimeMinutes, dtype: float64

### Q2.2

Merge the `title_ratings` `DataFrame` with the `title_basics` `DataFrame` by joining on the `tconst` column. How many titles are present in the `title_basics` `DataFrame` but not in the `title_ratings` `DataFrame`? Store the merged `DataFrame` as `merged_df`.

**Hint:** Recall that because of the genre splitting, the number of titles is not equal to the number of rows.

In [48]:
n_titles_basics = title_basics['tconst'].nunique()
# ... # YOUR CODE HERE

merged_df = title_basics.merge(title_ratings,on='tconst')
# ... # YOUR CODE HERE
print(merged_df.head())

n_titles_merged = merged_df['tconst'].nunique()
# ... # YOUR CODE HERE

print(f"\nNumber of titles in basics but not in ratings: {n_titles_basics - n_titles_merged}")

      tconst titleType                   title  isAdult  year  runtimeMinutes  \
0  tt0000001     short              Carmencita        0  1894             1.0   
1  tt0000001     short              Carmencita        0  1894             1.0   
2  tt0000002     short  Le clown et ses chiens        0  1892             5.0   
3  tt0000002     short  Le clown et ses chiens        0  1892             5.0   
4  tt0000003     short            Poor Pierrot        0  1892             5.0   

        genres  averageRating  numVotes  
0  Documentary            5.7      2184  
1        Short            5.7      2184  
2    Animation            5.5       305  
3        Short            5.5       305  
4    Animation            6.4      2267  

Number of titles in basics but not in ratings: 9913


### Q2.3

Using the `merged_df` `DataFrame` and plotly express, create an interactive scatter plot of the `runtimeMinutes` vs. `numVotes` for movies in the `merged_df` `DataFrame`.
Color the points by the `year` of the movie and add a title and axis labels to the plot. Also, make sure the movie title is visible when hovering over the
data points.

**Note:** To make the data easier to visualize, we take a sample of just 2000 movies. That's why you may not see your favorites on this plot. It's important not to change the random state as you'll end up getting different results for the following questions.

In [49]:
sampled_df = merged_df[merged_df['titleType'] == 'movie'].sample(n=2000, random_state=SEED)

# YOUR CODE HERE
fig = px.scatter(
    sampled_df,
    x='runtimeMinutes',
    y='numVotes',
    color='year',
    hover_data=['title'],
    title='Movie Runtime vs Number of Votes',
    labels={
        'runtimeMinutes': 'Runtime (Minutes)',
        'numVotes': 'Number of Votes',
        'year': 'Year'
    }
)
fig.show()

### Q2.4

Describe any trends you see in the plot above.

[YOUR ANSWER HERE] Most movies with good votes have runtime ranges from 50 to 150 minutes.

### Q2.5
Which two movies in the plot received the most votes and had the longest runtime, respectively? When were they each released?

[YOUR ANSWER HERE] Most votes: Peter Pan (1953) and Rebecca (1940). Longest runtime: The Mystery River (1928) and The Black Coin (1936)


# Part 3: Finding the perfect movie

Aakarsh has spent his whole summer brainrotting and doomscrolling, so now his attention span is COOKED. He wants to pick a movie to watch tonight but wants to make sure it isn't so long he gets bored. He decides to construct a Brainrot Score (BRS) to help him find the perfect movie:

$$BRS = \frac{\text{averageRating}}{\sqrt{\text{runtimeMinutes}}}$$

He also wants to make sure the following criteria hold:
- The title should be a *movie* made in 1980 or later.
- It must have at least 10000 votes.
- It must be in the `History`, `Thriller`, or `Comedy` genres.

Can you help Aakarsh out by finding the 3 best movies by BRS in each of his preferred genres?

In [50]:
# YOUR CODE HERE

merged_df['BRS'] = merged_df['averageRating'] / np.sqrt(merged_df['runtimeMinutes'])

# 筛选符合条件的电影
filtered_df = merged_df[
    (merged_df['titleType'] == 'movie') & 
    (merged_df['year'] >= 1980) & 
    (merged_df['numVotes'] >= 10000) & 
    (merged_df['genres'].isin(['History', 'Thriller', 'Comedy']))
]

# 按类型分组，每组取 BRS 最高的 3 部
result = filtered_df.groupby('genres').apply(
    lambda x: x.nlargest(3, 'BRS')
).reset_index(drop=True)

# 显示结果
print(result[['genres', 'title', 'year', 'BRS', 'averageRating', 'runtimeMinutes', 'numVotes']])

   genres           title  year       BRS  averageRating  runtimeMinutes  \
0  Comedy  Kate & Leopold  2001  0.589168            6.4           118.0   

   numVotes  
0     92821  


/tmp/ipykernel_1763/763243391.py:14: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

